In [1]:
import os
from os import listdir, makedirs
from os.path import join, exists, expanduser
from tqdm import tqdm

import random
import numpy as np
import pandas as pd

import datetime as dt
import time

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline

import cv2

import keras

from keras.callbacks import TensorBoard

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

from mpl_toolkits.axes_grid1 import ImageGrid

from keras.utils.np_utils import to_categorical

from keras.models import Sequential
from keras.models import Model


from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.layers import Dense, GlobalAveragePooling2D
from keras import optimizers
from keras.optimizers import RMSprop
from keras.callbacks import ReduceLROnPlateau


from keras import backend as K

# from keras.applications import vgg16, resnet50, mobilenet
# from keras.applications.vgg16 import VGG16
# from keras.applications.resnet50 import ResNet50
# from keras.applications import xception
# from keras.applications import inception_v3

from keras.applications.inception_v3 import InceptionV3

from keras.applications.vgg16 import preprocess_input, decode_predictions

from keras.optimizers import SGD




np.random.seed(2)
sns.set(style='white', context='notebook', palette='deep')

C:\Users\Andromeda\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def show_pred(preds, Y, val_breed, index, seq, ran):
    leng = len(preds)
    if seq:
        for i in range(index):
            if ran:
                index = random.randint(0, leng) 
            _, imagenet_class_name, prob = decode_predictions(preds, top=1)[index][0]
            plt.title("Original: " + val_breed[Y[index]] + "\nPrediction: " + imagenet_class_name)
            plt.imshow(X_train[index])
            plt.show()
    else:
            _, imagenet_class_name, prob = decode_predictions(preds, top=1)[index][0]
            plt.title("Original: " + val_breed[Y[index]] + "\nPrediction: " + imagenet_class_name)
            plt.imshow(X_train[index])
            plt.show()
        
def accuracy_func(preds, Y, val_breed):
    leng = len(preds)
    count = 0;
    for i in range(leng):
        _, imagenet_class_name, prob = decode_predictions(preds, top=1)[i][0]
        if val_breed[Y[i]] == imagenet_class_name:
            count+=1
    accuracy = (count/leng)*100
    
    print("Accuracy: ", accuracy)
    return accuracy

In [7]:
cache_dir = expanduser(join('~', '.keras'))
if not exists(cache_dir):
    makedirs(cache_dir)
models_dir = join(cache_dir, 'models')
if not exists(models_dir):
    makedirs(models_dir)

In [8]:
# #Load the VGG model
# vgg_model = vgg16.VGG16(weights='imagenet')
 
# #Load the Inception_V3 model
# inception_model = inception_v3.InceptionV3(weights='imagenet')
 
# #Load the ResNet50 model
# resnet_model = resnet50.ResNet50(weights='imagenet')
 
# #Load the MobileNet model
# mobilenet_model = mobilenet.MobileNet(weights='imagenet')

In [9]:
training_path = 'data/train'
validation_path = 'data/val'
testing_path = 'data/test'
batch_size = 32
target_size=(224, 224)
norm = 255.0
class_mode='categorical'

In [10]:
train_datagen = ImageDataGenerator(
        rescale=1./norm,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

validation_datagen = ImageDataGenerator(rescale=1./norm)
test_datagen = ImageDataGenerator(rescale=1./norm)

train_generator = train_datagen.flow_from_directory(
        training_path,
        target_size=target_size,
        batch_size=batch_size,
        class_mode=class_mode)

validation_generator = validation_datagen.flow_from_directory(
        validation_path,
        target_size=target_size,
        batch_size=batch_size,
        class_mode=class_mode)

test_generator = test_datagen.flow_from_directory(
        testing_path,
        target_size=target_size,
        batch_size=batch_size,
        class_mode=class_mode)

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [11]:
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

In [16]:
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(2, activation='softmax')(x)


In [17]:
# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [18]:
# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
    print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
    layer.trainable = False
for layer in model.layers[249:]:
    layer.trainable = True



0 input_1
1 conv2d_1
2 batch_normalization_1
3 activation_1
4 conv2d_2
5 batch_normalization_2
6 activation_2
7 conv2d_3
8 batch_normalization_3
9 activation_3
10 max_pooling2d_1
11 conv2d_4
12 batch_normalization_4
13 activation_4
14 conv2d_5
15 batch_normalization_5
16 activation_5
17 max_pooling2d_2
18 conv2d_9
19 batch_normalization_9
20 activation_9
21 conv2d_7
22 conv2d_10
23 batch_normalization_7
24 batch_normalization_10
25 activation_7
26 activation_10
27 average_pooling2d_1
28 conv2d_6
29 conv2d_8
30 conv2d_11
31 conv2d_12
32 batch_normalization_6
33 batch_normalization_8
34 batch_normalization_11
35 batch_normalization_12
36 activation_6
37 activation_8
38 activation_11
39 activation_12
40 mixed0
41 conv2d_16
42 batch_normalization_16
43 activation_16
44 conv2d_14
45 conv2d_17
46 batch_normalization_14
47 batch_normalization_17
48 activation_14
49 activation_17
50 average_pooling2d_2
51 conv2d_13
52 conv2d_15
53 conv2d_18
54 conv2d_19
55 batch_normalization_13
56 batch_norma

In [19]:
# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate



sgd = optimizers.Adam()
# sgd = optimizers.SGD()
# sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
# sgd = optimizer=SGD(lr=0.0001, momentum=0.9)

model.compile(sgd, loss='categorical_crossentropy', metrics=["accuracy"])

# model.compile(loss='mean_squared_error', optimizer='sgd')

In [20]:
model_dir = 'outputs/models/'
log_file = "outputs/logs"

model_file = model_dir+"weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"

In [21]:
checkpoint = keras.callbacks.ModelCheckpoint(model_file, monitor='val_acc', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)

In [24]:
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto', baseline=None)

In [25]:
tensorboard = keras.callbacks.TensorBoard(log_dir=log_file, histogram_freq=0, batch_size=32, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None)
tensorboard.set_model(model) 

In [26]:
callbacks_list = [checkpoint, tensorboard, early_stopping]
# callbacks_list = [checkpoint, tensorboard]

In [27]:
# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
history = model.fit_generator(
    train_generator,
    epochs=15,
    validation_data=validation_generator,
    verbose=1,
    callbacks=callbacks_list)

Epoch 1/15
163/163 [==============================] - ETA: 49:25 - loss: 0.6451 - acc: 0.71 - ETA: 42:53 - loss: 0.7946 - acc: 0.68 - ETA: 40:52 - loss: 1.1742 - acc: 0.58 - ETA: 37:43 - loss: 0.9037 - acc: 0.67 - ETA: 36:00 - loss: 0.7315 - acc: 0.73 - ETA: 34:57 - loss: 0.7148 - acc: 0.75 - ETA: 34:22 - loss: 0.7961 - acc: 0.76 - ETA: 33:54 - loss: 0.8010 - acc: 0.78 - ETA: 33:38 - loss: 0.7438 - acc: 0.79 - ETA: 32:51 - loss: 0.7128 - acc: 0.80 - ETA: 32:14 - loss: 0.6614 - acc: 0.81 - ETA: 31:30 - loss: 0.6188 - acc: 0.82 - ETA: 30:53 - loss: 0.5939 - acc: 0.82 - ETA: 30:20 - loss: 0.5855 - acc: 0.83 - ETA: 29:50 - loss: 0.5549 - acc: 0.83 - ETA: 29:22 - loss: 0.5637 - acc: 0.83 - ETA: 28:55 - loss: 0.5387 - acc: 0.84 - ETA: 28:30 - loss: 0.5144 - acc: 0.84 - ETA: 28:07 - loss: 0.5136 - acc: 0.84 - ETA: 27:45 - loss: 0.5303 - acc: 0.84 - ETA: 27:27 - loss: 0.5103 - acc: 0.85 - ETA: 27:08 - loss: 0.4925 - acc: 0.85 - ETA: 26:48 - loss: 0.4789 - acc: 0.86 - ETA: 26:31 - loss: 0.4687 

163/163 [==============================] - ETA: 33:43 - loss: 0.0871 - acc: 0.96 - ETA: 32:59 - loss: 0.2087 - acc: 0.95 - ETA: 33:39 - loss: 0.1673 - acc: 0.95 - ETA: 33:35 - loss: 0.1423 - acc: 0.96 - ETA: 34:17 - loss: 0.1537 - acc: 0.95 - ETA: 35:08 - loss: 0.1507 - acc: 0.95 - ETA: 34:50 - loss: 0.1423 - acc: 0.95 - ETA: 34:21 - loss: 0.1306 - acc: 0.96 - ETA: 33:58 - loss: 0.1229 - acc: 0.96 - ETA: 33:20 - loss: 0.1202 - acc: 0.96 - ETA: 32:51 - loss: 0.1210 - acc: 0.96 - ETA: 32:28 - loss: 0.1242 - acc: 0.96 - ETA: 32:14 - loss: 0.1236 - acc: 0.95 - ETA: 31:47 - loss: 0.1230 - acc: 0.95 - ETA: 31:35 - loss: 0.1190 - acc: 0.96 - ETA: 31:29 - loss: 0.1190 - acc: 0.95 - ETA: 31:18 - loss: 0.1174 - acc: 0.95 - ETA: 31:00 - loss: 0.1258 - acc: 0.95 - ETA: 30:40 - loss: 0.1208 - acc: 0.95 - ETA: 30:37 - loss: 0.1322 - acc: 0.95 - ETA: 30:21 - loss: 0.1398 - acc: 0.95 - ETA: 30:26 - loss: 0.1353 - acc: 0.95 - ETA: 30:21 - loss: 0.1303 - acc: 0.95 - ETA: 30:05 - loss: 0.1295 - acc: 0.95

163/163 [==============================] - ETA: 33:28 - loss: 0.0643 - acc: 1.00 - ETA: 32:38 - loss: 0.0421 - acc: 1.00 - ETA: 31:14 - loss: 0.0454 - acc: 0.98 - ETA: 30:26 - loss: 0.0490 - acc: 0.98 - ETA: 29:55 - loss: 0.0505 - acc: 0.98 - ETA: 29:23 - loss: 0.0732 - acc: 0.97 - ETA: 29:16 - loss: 0.0721 - acc: 0.97 - ETA: 29:34 - loss: 0.0787 - acc: 0.96 - ETA: 29:48 - loss: 0.0817 - acc: 0.96 - ETA: 29:40 - loss: 0.0871 - acc: 0.96 - ETA: 29:41 - loss: 0.0858 - acc: 0.96 - ETA: 29:36 - loss: 0.0828 - acc: 0.96 - ETA: 29:20 - loss: 0.0834 - acc: 0.96 - ETA: 28:58 - loss: 0.0786 - acc: 0.97 - ETA: 28:42 - loss: 0.0753 - acc: 0.97 - ETA: 28:27 - loss: 0.0750 - acc: 0.97 - ETA: 28:10 - loss: 0.0769 - acc: 0.97 - ETA: 27:57 - loss: 0.0754 - acc: 0.97 - ETA: 27:38 - loss: 0.0846 - acc: 0.96 - ETA: 27:31 - loss: 0.0830 - acc: 0.97 - ETA: 27:18 - loss: 0.0887 - acc: 0.97 - ETA: 27:05 - loss: 0.0870 - acc: 0.97 - ETA: 26:55 - loss: 0.0884 - acc: 0.96 - ETA: 26:44 - loss: 0.0887 - acc: 0.96

 89/163 [===============>..............] - ETA: 31:16 - loss: 0.1218 - acc: 0.96 - ETA: 29:38 - loss: 0.0816 - acc: 0.98 - ETA: 30:24 - loss: 0.0669 - acc: 0.98 - ETA: 30:37 - loss: 0.0860 - acc: 0.96 - ETA: 30:06 - loss: 0.0845 - acc: 0.96 - ETA: 29:55 - loss: 0.0819 - acc: 0.96 - ETA: 29:35 - loss: 0.0762 - acc: 0.96 - ETA: 29:12 - loss: 0.0747 - acc: 0.96 - ETA: 28:55 - loss: 0.0703 - acc: 0.96 - ETA: 28:48 - loss: 0.0710 - acc: 0.96 - ETA: 28:34 - loss: 0.0684 - acc: 0.97 - ETA: 28:10 - loss: 0.0651 - acc: 0.97 - ETA: 27:52 - loss: 0.0653 - acc: 0.97 - ETA: 27:34 - loss: 0.0707 - acc: 0.97 - ETA: 27:22 - loss: 0.0680 - acc: 0.97 - ETA: 27:11 - loss: 0.0670 - acc: 0.97 - ETA: 26:58 - loss: 0.0696 - acc: 0.97 - ETA: 26:38 - loss: 0.0728 - acc: 0.97 - ETA: 26:22 - loss: 0.0706 - acc: 0.97 - ETA: 26:08 - loss: 0.0681 - acc: 0.97 - ETA: 25:53 - loss: 0.0798 - acc: 0.97 - ETA: 25:38 - loss: 0.0829 - acc: 0.96 - ETA: 25:23 - loss: 0.0798 - acc: 0.96 - ETA: 25:08 - loss: 0.0796 - acc: 0.96

KeyboardInterrupt: 

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [28]:
filepath = 'outputs/models/weights-improvement-03-0.62.hdf5'
model = keras.models.load_model(filepath)

In [ ]:
# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
history = model.fit_generator(
    train_generator,
    epochs=15,
    validation_data=validation_generator,
    verbose=1,
    callbacks=callbacks_list)

Epoch 1/15
163/163 [==============================] - ETA: 47:57 - loss: 0.1382 - acc: 0.93 - ETA: 41:37 - loss: 0.1177 - acc: 0.95 - ETA: 37:26 - loss: 0.0812 - acc: 0.96 - ETA: 35:01 - loss: 0.0979 - acc: 0.96 - ETA: 33:28 - loss: 0.0864 - acc: 0.96 - ETA: 32:12 - loss: 0.0855 - acc: 0.96 - ETA: 31:22 - loss: 0.0810 - acc: 0.96 - ETA: 30:43 - loss: 0.0847 - acc: 0.96 - ETA: 30:03 - loss: 0.0947 - acc: 0.96 - ETA: 29:35 - loss: 0.0903 - acc: 0.96 - ETA: 29:10 - loss: 0.0850 - acc: 0.96 - ETA: 28:43 - loss: 0.0803 - acc: 0.97 - ETA: 28:23 - loss: 0.0757 - acc: 0.97 - ETA: 28:04 - loss: 0.0784 - acc: 0.97 - ETA: 27:43 - loss: 0.0808 - acc: 0.96 - ETA: 27:27 - loss: 0.0795 - acc: 0.97 - ETA: 27:12 - loss: 0.0790 - acc: 0.97 - ETA: 26:53 - loss: 0.0772 - acc: 0.97 - ETA: 26:39 - loss: 0.0780 - acc: 0.97 - ETA: 26:25 - loss: 0.0754 - acc: 0.97 - ETA: 26:09 - loss: 0.0907 - acc: 0.96 - ETA: 25:55 - loss: 0.0898 - acc: 0.96 - ETA: 25:42 - loss: 0.0868 - acc: 0.97 - ETA: 25:27 - loss: 0.0837 

163/163 [==============================] - ETA: 28:30 - loss: 0.1202 - acc: 0.93 - ETA: 29:43 - loss: 0.0922 - acc: 0.93 - ETA: 28:56 - loss: 0.1090 - acc: 0.94 - ETA: 28:34 - loss: 0.0831 - acc: 0.96 - ETA: 28:19 - loss: 0.0850 - acc: 0.96 - ETA: 28:23 - loss: 0.0750 - acc: 0.96 - ETA: 28:04 - loss: 0.0983 - acc: 0.96 - ETA: 27:50 - loss: 0.1022 - acc: 0.96 - ETA: 27:32 - loss: 0.0955 - acc: 0.96 - ETA: 27:27 - loss: 0.0928 - acc: 0.96 - ETA: 27:13 - loss: 0.0927 - acc: 0.96 - ETA: 26:58 - loss: 0.0871 - acc: 0.96 - ETA: 27:18 - loss: 0.0821 - acc: 0.96 - ETA: 27:33 - loss: 0.0848 - acc: 0.96 - ETA: 27:18 - loss: 0.0829 - acc: 0.96 - ETA: 27:05 - loss: 0.0801 - acc: 0.96 - ETA: 26:52 - loss: 0.0769 - acc: 0.96 - ETA: 26:39 - loss: 0.0733 - acc: 0.96 - ETA: 26:36 - loss: 0.0724 - acc: 0.97 - ETA: 26:41 - loss: 0.0702 - acc: 0.97 - ETA: 26:34 - loss: 0.0767 - acc: 0.97 - ETA: 26:29 - loss: 0.0737 - acc: 0.97 - ETA: 26:17 - loss: 0.0732 - acc: 0.97 - ETA: 26:08 - loss: 0.0727 - acc: 0.97

 60/163 [==========>...................] - ETA: 27:17 - loss: 0.0367 - acc: 1.00 - ETA: 27:16 - loss: 0.0450 - acc: 0.98 - ETA: 27:15 - loss: 0.0437 - acc: 0.98 - ETA: 27:01 - loss: 0.0780 - acc: 0.98 - ETA: 26:59 - loss: 0.0773 - acc: 0.97 - ETA: 26:53 - loss: 0.0997 - acc: 0.96 - ETA: 26:38 - loss: 0.0908 - acc: 0.96 - ETA: 26:29 - loss: 0.0879 - acc: 0.96 - ETA: 26:21 - loss: 0.0815 - acc: 0.96 - ETA: 26:08 - loss: 0.0829 - acc: 0.96 - ETA: 26:00 - loss: 0.0796 - acc: 0.96 - ETA: 25:50 - loss: 0.0939 - acc: 0.96 - ETA: 25:39 - loss: 0.1014 - acc: 0.95 - ETA: 25:31 - loss: 0.1089 - acc: 0.95 - ETA: 25:19 - loss: 0.1037 - acc: 0.96 - ETA: 25:09 - loss: 0.0995 - acc: 0.96 - ETA: 25:00 - loss: 0.0958 - acc: 0.96 - ETA: 24:48 - loss: 0.0939 - acc: 0.96 - ETA: 24:38 - loss: 0.0902 - acc: 0.96 - ETA: 24:29 - loss: 0.0874 - acc: 0.96 - ETA: 24:18 - loss: 0.0946 - acc: 0.96 - ETA: 24:08 - loss: 0.0949 - acc: 0.96 - ETA: 23:59 - loss: 0.0916 - acc: 0.96 - ETA: 23:47 - loss: 0.0891 - acc: 0.96

In [29]:
preds = model.predict_generator(test_generator, verbose=1)

20/20 [==============================] - ETA: 5: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 56s - ETA: 42 - ETA: 28 - ETA: 14 - 277s 14s/step


In [30]:
categorical_accuracy = keras.metrics.categorical_accuracy(test_generator.classes, preds)

In [31]:
result = model.evaluate_generator(generator=test_generator, verbose=1)

20/20 [==============================] - ETA: 5: - ETA: 4: - ETA: 4: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 53s - ETA: 40 - ETA: 26 - ETA: 13 - 264s 13s/step


In [32]:
print("%s%.2f%s"% ("Accuracy: ", result[1]*100, "%"))
print("%s%.2f"% ("Loss: ", result[0]))

Accuracy: 68.91%
Loss: 1.19


In [ ]:
def get_class_values(gen):
    vals = []
    for i in gen:
        vals.append(gen[i])
    return vals

def get_classes(gen):
    rev_gen = {}
    for i in gen:
        rev_gen[gen[i]] = i
    return rev_gen

def find_max(preds_i):
    num = -1
    index = -1
    for i in range(len(preds_i)):
        if preds_i[i]>num:
            num = preds_i[i]
            index = i
    return index  

def get_label(cls, val):
    return cls[val]

In [ ]:
vals = get_class_values(test_generator.class_indices)
cls = get_classes(test_generator.class_indices)

In [ ]:
total = 0
acc = 0
direc = "data\\pre\\test"
files = test_generator.filenames
for i in range(len(preds)):
    cls_index = find_max(preds[i])
    pred_label = cls[cls_index]
    org_label = cls[test_generator.classes[i]]
    file_name = os.path.join(direc, files[i])
    if org_label==pred_label:
        acc +=1
    if total%10==0:
        result = "Wrong!!!\n"
        if org_label==pred_label:
            result = "Right!!!\n"
        title_text = "Original: "+ org_label+"\nPredicted: "+ pred_label+"\nFile: "+ file_name
        title_text=result+ title_text
        
        img = cv2.imread(file_name, 3)
        if org_label==pred_label:
            plt.title(title_text, color='blue')
        else:
            plt.title(title_text, color='red')
        plt.imshow(img)
        plt.show()
    
    total+=1
accuracy = (acc/total)*100
print("Accuracy: ", accuracy)